Установка модуля для обработки описаний

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Импорт модулей

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json
import os
import sys

Подгружаем гугл диск и файл с него

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/dump.json', "r") as read_file:
    json_file = read_file.readlines()

Заполняем список описаниями

In [ ]:
sentences = []

In [ ]:
for line in json_file:
  if line.startswith('{'):
    value = json.loads(line[0:(line.rfind('}')+1)])['description']
    sentences.append(value)

In [ ]:
isbn_code = []
for line in json_file:
  if line.startswith('{'):
    value = json.loads(line[0:(line.rfind('}')+1)])['ISBN']
    isbn_code.append(value)


Преобразуем описания в векторы

In [ ]:
model_name = 'sentence-transformers/stsb-xlm-r-multilingual'

model = SentenceTransformer(model_name)

sentence_vecs = model.encode(sentences[:10000])
sentence_vecs.shape

(10000, 768)

Переводим векторы в формат подходящий для базы данных Pipecone и сохранем в JSON файл.

In [ ]:
sentence_dict = {
    'vectors' : []
}

In [ ]:

for i in range(10000):
  sentence_dict['vectors'].append({'id': isbn_code[i],
                                   'values': sentence_vecs[i]})

In [ ]:
import json
from json import JSONEncoder
import numpy

In [ ]:
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

In [ ]:
with open("/content/drive/MyDrive/vectors.json", "w") as outfile:
    json.dump(sentence_dict, outfile, cls=NumpyArrayEncoder)